### Uso de Spark Mlib y prueba del modelo con distintos algoritmos de clasificación para hacer el análisis de sentimiento de tweets.

In [1]:
# imports y configuraciones necesarias
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, NaiveBayes, RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, CountVectorizer, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

import pandas as pd

import re
import string
import unicodedata

import nltk
from nltk.corpus import stopwords
# cargamos las stopwords para cada idioma
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')


conf = (SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate())  

sc = SparkContext.getOrCreate(conf=conf)

#### Funciones necesarias para la preparación de los datos y el análisis

In [2]:
# Función de carga de datos desde MongoDB de los datos almacenados desde Apache Nifi, tanto en inglés como español
def carga_datos_mongo():
    # conectamos con las tablas de Mongo desde donde cargamos los datos
    df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()
    df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
        .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()
    
    # nos quedamos solo con el atributo texto que es la información que nos interesa de cada dataset
    df_text_english = df_mongo_english[['text']]
    df_text_spanish = df_mongo_spanish[['text']]
    
    return df_text_english, df_text_spanish


# Función de carga de datos desde los csv generados con anterioridad con registros ya con el sentimiento anotado
def carga_datos_csv(csv):
    schema_csv = StructType([ 
        StructField("text", StringType(), True),
        StructField("sentiment", IntegerType(), True)
    ])
    
    # cargamos los csv
    if(csv=='english_full'): fichero = './data/df_result_english.csv'
    elif(csv=='spanish_full'): fichero = './data/df_result_spanish.csv'
    elif(csv=='english_neutro'): fichero = './data/df_result_english_neutral.csv'
    elif(csv=='spanish_neutro'): fichero = './data/df_result_spanish_neutral.csv'
    elif(csv=='english_noNeutro'): fichero = './data/df_result_english_noNeutral.csv'
    elif(csv=='spanish_noNeutro'): fichero = './data/df_result_spanish_noNeutral.csv'
    
    df_csv = sqlContext.\
        read.format("com.databricks.spark.csv").\
        option("header", "true").\
        option("inferschema", "true").\
        option("mode", "DROPMALFORMED").\
        schema(schema_csv).\
        load(fichero).\
        cache()
    
    return df_csv


# Funciones de visualización de DFs para ver las columnas, el número de registros o dimensiones, y el recuento
# de valores del atributo sentimiento en caso de tener la columna.
def visualizar_datos_csv(df):
    print("Columnas del dataframe: ", df.columns)
    print("Numero de registros = %d" % df.count())
    print("\n")
    print(df.limit(10).toPandas())
    print("\n")
    recuento_sentiment = df.select('sentiment').groupBy("sentiment").count().show()
    print("\n")

def visualizar_datos_mongo(df):
    df_pandas = df.toPandas()
    print("num_rows: %d\tColumnas: %d\n" % (df_pandas.shape[0], df_pandas.shape[1]) )
    print("Columnas:\n", list(df_pandas.columns))
    print(df_pandas.head(10))
    print("\n")


# función para eliminar palabras que no queramos analizar
def eliminar_stopwords(texto, palabras_eliminar):
    tok = nltk.tokenize
    palabras = tok.word_tokenize(texto)
        
    palabras_salida = []
        
    for palabra in palabras:
        if palabra not in palabras_eliminar:
            palabras_salida.append(palabra)
        
    salida = ""
    for i in range(len(palabras_salida)):
        if palabras_salida[i] in string.punctuation:
            salida = salida.strip()+palabras_salida[i] + " "
        else:
            salida += palabras_salida[i] + " "

    return salida


# Funciones de limpieza de los tweets
def limpieza_tweets_spanish(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 
    # eliminar acentos
    tweet = [''.join((c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')) for s in tweet]
        
    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, spanish_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)

# a los datos en inglés le aplicamos sus stopwords correspondientes y no les quitamos los acentos
def limpieza_tweets_english(tokens : list) -> list:
    tweet = [re.sub('  +', ' ', s).strip() for s in tokens]
    tweet = [re.sub(r'http\S+', '', s) for s in tweet]  
    tweet = [re.sub(r'@[\S]+', '', s) for s in tweet]
    tweet = [re.sub(r'#(\S+)', r' \1 ', s) for s in tweet]
    tweet = [re.sub(r'\brt\b', '', s) for s in tweet]
    tweet = [re.sub(r'\.{2,}', ' ', s) for s in tweet]
    tweet = [re.sub(r'\s+', ' ', s) for s in tweet]
    tweet = [re.sub('','',s).lower() for s in tweet]
    
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = [re.sub(r'(.)\1+', r'\1\1', s) for s in tweet]
    # remover - & '
    tweet = [re.sub(r'(-|\')', '', s) for s in tweet] 

    # reemplazar emojis
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = [emoji_pattern.sub(r' ', s) for s in tweet]
    tweet = [re.sub("[^A-Za-z]+$",'',s) for s in tweet]
    tweet = [re.sub("^[^A-Za-z]+",'',s) for s in tweet]
    tweet = [re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',s) for s in tweet]  
    
    # remover stopwords
    tweet = [eliminar_stopwords(s, english_stopwords) for s in tweet]

    filtered = filter(None, tweet)
    
    return list(filtered)


# Función de preprocesado de los datos
def preprocesado_dataframe(df1, df2):
    # primero usamos tokenizer y vamos a partir los tweets por palabras
    tokenizer = Tokenizer(inputCol = "text", outputCol = "token")

    df_tokens_english = tokenizer.transform(df1)
    df_tokens_spanish = tokenizer.transform(df2)  
    
    # usamos las funciones de limpieza y preprocesado con ambos DFs ya con los textos divididos en tokens
    limpiezaUDF_english = F.udf(limpieza_tweets_english, ArrayType(StringType()))
    limpiezaUDF_spanish = F.udf(limpieza_tweets_spanish, ArrayType(StringType()))

    df_tokens_english = df_tokens_english.withColumn("tokens_clean", limpiezaUDF_english(df_tokens_english["token"]))
    df_tokens_spanish = df_tokens_spanish.withColumn("tokens_clean", limpiezaUDF_spanish(df_tokens_spanish["token"]))

    df_tokens_english = df_tokens_english.drop("token")
    df_tokens_spanish = df_tokens_spanish.drop("token")

    df_tokens_english_clean = df_tokens_english.where(F.size(F.col("tokens_clean")) > 0)
    df_tokens_spanish_clean = df_tokens_spanish.where(F.size(F.col("tokens_clean")) > 0)
    
    return df_tokens_english_clean, df_tokens_spanish_clean


# Función evaluación de modelo
def evaluar_modelo(metric, predCol, labelCol, pred1, pred2):
    evaluator = MulticlassClassificationEvaluator(metricName=metric, predictionCol=predCol, labelCol=labelCol)

    eval_eng_clean = evaluator.evaluate(pred1)
    print("-------------------------------------------------------")
    print("Medidas de rendimiento datos en inglés")
    print("-------------------------------------------------------")
    print("F1-score = %f" % eval_eng_clean)
    print("\n")
    
    eval_spa_clean = evaluator.evaluate(pred2)
    print("-------------------------------------------------------")
    print("Medidas de rendimiento datos en español")
    print("-------------------------------------------------------")
    print("F1-score = %f" % eval_spa_clean)
    print("\n") 


# Función para ver los mejores parámetros de un modelo al usar cross-validation
def mejores_parametros(model1, model2):
    bestPipeline_eng = model1.bestModel
    bestPipeline_spa = model2.bestModel

    bestVectorizer_eng = bestPipeline_eng.stages[0]
    bestVectorizer_spa = bestPipeline_spa.stages[0]

    bestParamsVect_eng = bestVectorizer_eng.extractParamMap()
    print ("Vectorizer parameters datos inglés:")
    for k in bestParamsVect_eng.keys():
        print ("  ", k, bestParamsVect_eng[k])
    
    bestParamsVect_spa = bestVectorizer_spa.extractParamMap()
    print("\n") 
    print ("Vectorizer parameters datos español:")
    for k in bestParamsVect_spa.keys():
        print ("  ", k, bestParamsVect_spa[k])

    bestModel_eng = bestPipeline_eng.stages[2]
    bestModel_spa = bestPipeline_spa.stages[2]

    bestParamsModel_eng = bestModel_eng.extractParamMap()
    print("\n") 
    print("Model parameters datos inglés:")
    for k in bestParamsModel_eng.keys():
        print("  ", k, bestParamsModel_eng[k])

    bestParamsModel_spa = bestModel_spa.extractParamMap()
    print("\n") 
    print("Model parameters datos español:")
    for k in bestParamsModel_spa.keys():
        print("  ", k, bestParamsModel_spa[k])

#### Cargamos los datos que pueden ser necesarios tanto para entrenar los modelos como para testearlos.

Los datos almacenados en MongoDB que no tienen sentimiento calculado, se pueden utilizar para probar luego el modelo a predecirlo.

Los datos que se cargan desde los csv generados con datos en inglés y español ya con sentimiento calculado, se van a usar para entrenar los modelos.

In [3]:
# cargamos los datos de MongoDB que podrían usarse para probar el modelo final
df_mongo_english, df_mongo_spanish = carga_datos_mongo()

In [4]:
visualizar_datos_mongo(df_mongo_english)

num_rows: 609133	Columnas: 1

Columnas:
 ['text']
                                                text
0                                    @EJFC26 Y Messi
1  RT @btsmoonchild64: These group photos deserve...
2                        @rehankkhanNDS Overacting *
3                              Pay day play day🤑🤑🤑🤑🤑
4                             @pandaeyed1 Thank you!
5  RT @liamyoung: Strange that Tony Blair has sud...
6   Hard work puts you where good luck can find you.
7  RT @xCiphxr: When creative kids try playing co...
8  RT @bonang_m: I’m working on one as we speak. ...
9  RT @akashbanerjee: After #PulwamaAttack, terro...




In [5]:
visualizar_datos_mongo(df_mongo_spanish)

num_rows: 153619	Columnas: 1

Columnas:
 ['text']
                                                text
0                                    @EJFC26 Y Messi
1     RT @MBelenAlegre: Hermoso y sensual viernes ❣️
2  RT @Foro_TV: Suman 47 muertos y 640 heridos po...
3  RT @revistaetcetera: .@lopezobrador_ dice que ...
4  Me retracto de mi respuesta , él gobierno se s...
5  Anda a saber si te esta eligiendo o sos lo úni...
6  RT @JCTrujilloCPCCS: Hay que propiciar una Con...
7                            @RicciuP Terraplanistas
8  RT @oriolguellipuig: Jeje jeje https://t.co/NU...
9  quienes son las veganarcas? no quiero ser part...




In [6]:
# cargamos los datos almacenados en ficheros csv con el sentimiento etiquetado para entrenar modelos
df_csv_english_full = carga_datos_csv('english_full')
df_csv_spanish_full = carga_datos_csv('spanish_full')
df_csv_english_neutro = carga_datos_csv('english_neutro')
df_csv_spanish_neutro = carga_datos_csv('spanish_neutro')
df_csv_english_noNeutro = carga_datos_csv('english_noNeutro')
df_csv_spanish_noNeutro = carga_datos_csv('spanish_noNeutro')

In [7]:
visualizar_datos_csv(df_csv_english_full)

Columnas del dataframe:  ['text', 'sentiment']
Numero de registros = 1759091


                                                text  sentiment
0                @VirginAmerica What @dhepburn said.          1
1  @VirginAmerica plus you've added commercials t...          2
2  @VirginAmerica I didn't today... Must mean I n...          1
3  "@VirginAmerica it's really aggressive to blas...          0
4  @VirginAmerica and it's a really big bad thing...          0
5    it's really the only bad thing about flying VA"          0
6  @VirginAmerica yes, nearly every time I fly VX...          2
7  @VirginAmerica Really missed a prime opportuni...          1
8    @virginamerica Well, I didn't…but NOW I DO! :-D          2
9  @VirginAmerica it was amazing, and arrived an ...          2


+---------+------+
|sentiment| count|
+---------+------+
|        1| 14424|
|        2|872815|
|        0|871852|
+---------+------+





In [8]:
# cogemos un sample del DF con datos en inglés ya que tiene muchos datos y da problemas de cómputo y tiempos
df_csv_english_sample = df_csv_english_full.sample(False, 0.1, 45)

In [9]:
visualizar_datos_csv(df_csv_english_sample)

Columnas del dataframe:  ['text', 'sentiment']
Numero de registros = 175818


                                                text  sentiment
0  @VirginAmerica amazing to me that we can't get...          0
1  @VirginAmerica View of downtown Los Angeles, t...          2
2  @VirginAmerica plz help me win my bid upgrade ...          1
3  @VirginAmerica I'm #elevategold for a good rea...          2
4  @VirginAmerica @ladygaga @carrieunderwood Juli...          1
5  @VirginAmerica I’m having trouble adding this ...          0
6  @VirginAmerica you have the absolute best team...          2
7  @VirginAmerica has flight number 276 from SFO ...          1
8  @VirginAmerica Another delayed flight? #liking...          0
9  @VirginAmerica Can you find us a flt out of LA...          1


+---------+-----+
|sentiment|count|
+---------+-----+
|        1| 1372|
|        2|87372|
|        0|87074|
+---------+-----+





In [10]:
visualizar_datos_csv(df_csv_spanish_full)

Columnas del dataframe:  ['text', 'sentiment']
Numero de registros = 46787


                                                text  sentiment
0  @PauladeLasHeras No te libraras de ayudar me/n...          1
1                          @marodriguezb Gracias MAR          2
2  Off pensando en el regalito Sinde, la que se v...          0
3  Conozco a alguien q es adicto al drama! Ja ja ...          2
4  Toca @crackoviadeTV3 . Grabación dl especial N...          2
5  Buen día todos! Lo primero mandar un abrazo gr...          2
6  Desde el escaño. Todo listo para empezar #endi...          2
7  Bdías. EM no se ira de puente. Si vosotros os ...          2
8  Un sistema económico q recorta dinero para pre...          2
9                  #programascambiados caca d ajuste          0


+---------+-----+
|sentiment|count|
+---------+-----+
|        1| 2927|
|        2|25392|
|        0|18468|
+---------+-----+





#### Limpieza de los textos antes de hacer el análisis de sentimiento.

Se limpiarán los textos de elementos y caracteres que no tengan importancia para el análisis a realizar. Además se sacarán los textos tokenizados.

In [11]:
# llamamos a la función que engloba el preprocesado de los datos, con la tokenización y la limpieza de tweets
df_tokens_english_clean, df_tokens_spanish_clean = preprocesado_dataframe(df_csv_english_sample,df_csv_spanish_full)

In [12]:
# visualizamos un grupo de datos para ver el correcto preprocesamiento del texto
df_tokens_english_clean.limit(20).toPandas()

,text,sentiment,tokens_clean
0,@VirginAmerica amazing to me that we can't get...,0,"[amazing , cant , get , cold , air , vents , v..."
1,"@VirginAmerica View of downtown Los Angeles, t...",2,"[view , downtown , los , angeles , hollywood ,..."
2,@VirginAmerica plz help me win my bid upgrade ...,1,"[plz , help , win , bid , upgrade , flight , l..."
3,@VirginAmerica I'm #elevategold for a good rea...,2,"[im , elevategold , good , reason , rock ]"
4,@VirginAmerica @ladygaga @carrieunderwood Juli...,1,"[julie , andrews , first , lady , gaga , wowd ..."
5,@VirginAmerica I’m having trouble adding this ...,0,"[im , trouble , adding , flight , wife , booke..."
6,@VirginAmerica you have the absolute best team...,2,"[absolute , best , team , customer , service ,..."
7,@VirginAmerica has flight number 276 from SFO ...,1,"[flight , number , sfo , cabo , san , lucas , ..."
8,@VirginAmerica Another delayed flight? #liking...,0,"[another , delayed , flight , likingyoulessand..."
9,@VirginAmerica Can you find us a flt out of LA...,1,"[find , us , flt , lax , sooner , midnight , m..."


In [13]:
df_tokens_spanish_clean.limit(20).toPandas()

,text,sentiment,tokens_clean
0,@PauladeLasHeras No te libraras de ayudar me/n...,1,"[libraras , ayudar , menos , besos , gracias ]"
1,@marodriguezb Gracias MAR,2,"[gracias , mar ]"
2,"Off pensando en el regalito Sinde, la que se v...",0,"[off , pensando , regalito , sinde , va , sgae..."
3,Conozco a alguien q es adicto al drama! Ja ja ...,2,"[conozco , alguien , q , adicto , drama , ja ,..."
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2,"[toca , grabacion , dl , especial , navideno m..."
5,Buen día todos! Lo primero mandar un abrazo gr...,2,"[buen , dia , primero , mandar , abrazo , gran..."
6,Desde el escaño. Todo listo para empezar #endi...,2,"[escano , listo , empezar , endiascomohoy , co..."
7,Bdías. EM no se ira de puente. Si vosotros os ...,2,"[bdias , em , ira , puente , si , vais , dejei..."
8,Un sistema económico q recorta dinero para pre...,2,"[sistema , economico , q , recorta , dinero , ..."
9,#programascambiados caca d ajuste,0,"[programascambiados , caca , d , ajuste ]"


#### Modelos de Machine Learning.

Vamos a generar los modelos de aprendizaje supervisado, utilizando distintos algoritmos igual que se realizó con la librería Scikit-learn.

Usaremos los datos en inglés y español después del preprocesado y limpieza realizado anteriormente.

Para los modelos, se hace uso de Pipeline igual que en Scikit-learn, que contendrá el flujo de transformers y estimators que son necesarios para el modelo.

Para evaluar el modelo se utilizará "MulticlassClassificationEvaluator", ya que es un problema de clasificación con más de 2 clases. La métrica que se tendrá en cuenta es F1-score.

In [14]:
# a partir del dataframe anterior, vamos a eliminar la columna texto en los datos que han pasado por la limpieza y
# preprocesado.
df_tokens_english_clean = df_tokens_english_clean.drop("text")
df_tokens_spanish_clean = df_tokens_spanish_clean.drop("text")

In [15]:
df_tokens_english_clean.limit(5).toPandas()

,sentiment,tokens_clean
0,0,"[amazing , cant , get , cold , air , vents , v..."
1,2,"[view , downtown , los , angeles , hollywood ,..."
2,1,"[plz , help , win , bid , upgrade , flight , l..."
3,2,"[im , elevategold , good , reason , rock ]"
4,1,"[julie , andrews , first , lady , gaga , wowd ..."


In [16]:
# cambiamos el nombre de la columna sentiment por label a los dataframes
df_english_clean = df_tokens_english_clean.withColumnRenamed("sentiment", "label").cache()
df_spanish_clean = df_tokens_spanish_clean.withColumnRenamed("sentiment", "label").cache()

In [17]:
# generación de conjunto de training y test de los dataframes
train_english, test_english = df_english_clean.randomSplit([0.9, 0.1], seed=12345)
train_spanish, test_spanish = df_spanish_clean.randomSplit([0.9, 0.1], seed=12345)

In [18]:
# vemos el recuento de registros en los conjuntos de train y test por cada clase de sentimiento
train_english.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 1205|
|    2|78215|
|    0|78116|
+-----+-----+



In [19]:
test_english.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  141|
|    2| 8716|
|    0| 8539|
+-----+-----+



In [20]:
train_spanish.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 2619|
|    2|22781|
|    0|16612|
+-----+-----+



In [21]:
test_spanish.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  308|
|    2| 2589|
|    0| 1856|
+-----+-----+



In [40]:
# generamos los pipeline para cada modelo con las etapas o pasos que correspondan.
# primero probamos con Hashing y logisticRegression como algoritmo
hashingTF = HashingTF(inputCol="tokens_clean", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline_lr = Pipeline(stages=[hashingTF, lr])

In [41]:
# entrenamos los modelos
model_eng_lr = pipeline_lr.fit(train_english)
model_spa_lr = pipeline_lr.fit(train_spanish)

In [42]:
# usamos los modelos para predecir usando los datos de test
pred_eng_lr = model_eng_lr.transform(test_english)
pred_spa_lr = model_spa_lr.transform(test_spanish)

In [43]:
print("Predictions:")

pred_eng_lr.filter(pred_eng_lr['prediction'] == 0) \
    .select("tokens_clean","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

Predictions:
+------------------------------+------------------------------+-----+----------+
|                  tokens_clean|                   probability|label|prediction|
+------------------------------+------------------------------+-----+----------+
|[yach , allah , tawne , saw...|[0.9999999745421377,2.54446...|    0|       0.0|
|[grumpy , tomorrow , hes , ...|[0.999999974241276,2.566668...|    0|       0.0|
|[boo , still , mia , printe...|[0.9999999538056606,4.61111...|    0|       0.0|
|[chickens , chickens , damn...|[0.9999999238062758,7.59345...|    0|       0.0|
|[pts , expired , made , prc...|[0.9999999049176175,9.50285...|    0|       0.0|
|[today , im , really , amaz...|[0.9999998019933557,7.29863...|    0|       0.0|
|[going , chumlees , chinese...|[0.9999997774096685,2.20251...|    0|       0.0|
|[ohgosh , hate , recession ...|[0.9999997621053274,2.33153...|    0|       0.0|
|[wow , feel , like , ish , ...|[0.9999997229447234,2.64365...|    0|       0.0|
|[seriouslyy , 

In [44]:
print("Predictions:")

pred_spa_lr.filter(pred_spa_lr['prediction'] == 0) \
    .select("tokens_clean","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

Predictions:
+------------------------------+------------------------------+-----+----------+
|                  tokens_clean|                   probability|label|prediction|
+------------------------------+------------------------------+-----+----------+
|[k , k , detuvieron , k , d...|[0.9999999987991242,9.12566...|    0|       0.0|
|[patxi , lopez , anuncia , ...|[0.999999998509266,1.268585...|    0|       0.0|
|[paramo , huye , preguntas ...|[0.9999999979516403,6.26268...|    0|       0.0|
|[sabiais , grinan , generad...|[0.9999999974993734,1.67380...|    0|       0.0|
|[muerto , heridos , graves ...|[0.9999999962963302,3.70298...|    0|       0.0|
|[explosion , gas , posible ...|[0.9999999939276418,6.05849...|    0|       0.0|
|[dicen , escuchamos , gemel...|[0.9999999923347495,5.98716...|    0|       0.0|
|[deficit , ciento , recesio...|[0.9999999920444488,3.35025...|    0|       0.0|
|[acusa , grinan , confundir...|[0.9999999898535853,9.83601...|    0|       0.0|
|[parece , grav

In [45]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_lr, pred_spa_lr)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.716218


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.758850




In [28]:
# Vamos a probar cambiando HashingVectorizer por CountVectorizer
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline_countVec = Pipeline(stages=[countVec, lr])

In [29]:
# entrenamos los modelos
model_eng_countVec = pipeline_countVec.fit(train_english)
model_spa_countVec = pipeline_countVec.fit(train_spanish)

In [30]:
# usamos los modelos para predecir usando los datos de test
pred_eng_countVec = model_eng_countVec.transform(test_english)
pred_spa_countVec = model_spa_countVec.transform(test_spanish)

In [31]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_countVec, pred_spa_countVec)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.727583


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.765104




In [32]:
# Vamos a probar a añadir después del vectorizador "countVectorizer" el uso de TD-IDF
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline_idf = Pipeline(stages=[countVec, idf, lr])

In [33]:
# entrenamos los modelos
model_eng_idf = pipeline_idf.fit(train_english)
model_spa_idf = pipeline_idf.fit(train_spanish)

In [34]:
# usamos los modelos para predecir usando los datos de test
pred_eng_idf = model_eng_idf.transform(test_english)
pred_spa_idf = model_spa_idf.transform(test_spanish)

In [35]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_idf, pred_spa_idf)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.738500


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.769939




#### Probar distintos algoritmos.

Vamos ahora a probar con el último pipeline generado, que se habían mejorado ligeramente los resultados, con los algoritmos de Naive Bayes y Random Forest.

In [36]:
# Algoritmo Naive Bayes
nb = NaiveBayes(smoothing=1, modelType="multinomial")

pipeline_nb = Pipeline(stages=[countVec, idf, nb])

In [37]:
# entrenamos los modelos
model_eng_nb = pipeline_nb.fit(train_english)
model_spa_nb = pipeline_nb.fit(train_spanish)

In [38]:
# usamos los modelos para predecir usando los datos de test
pred_eng_nb = model_eng_nb.transform(test_english)
pred_spa_nb = model_spa_nb.transform(test_spanish)

In [39]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_nb, pred_spa_nb)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.734045


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.762048




In [46]:
# Algoritmo Random Forest
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 20, \
                            maxDepth = 4, \
                            maxBins = 32)
pipeline_rf = Pipeline(stages=[countVec, idf, rf])

In [47]:
# entrenamos los modelos
model_eng_rf = pipeline_rf.fit(train_english)

In [48]:
model_spa_rf = pipeline_rf.fit(train_spanish)

In [49]:
# usamos los modelos para predecir usando los datos de test
pred_eng_rf = model_eng_rf.transform(test_english)
pred_spa_rf = model_spa_rf.transform(test_spanish)

In [50]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_rf, pred_spa_rf)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.553909


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.384633




#### Selección de modelos y ajuste de los hiperparámetros.

Hasta ahora los mejores resultados se han conseguido usando el algoritmo de LogisticRegression y Naive Bayes. Ahora vamos a probar a realizar ajuste de hiperparámetros usando CrossValidator y TrainValidationSplit.
Para ver los mejores hiperparámetros para el modelo se usan conjuntos de validación. 

Durante el proceso de validación, el conjunto de datos se divide en dos (entrenamiento y validación). El conjunto de entrenamiento se utiliza para entrenar modelos con cada uno de los posibles conjuntos de valores de los hiperparámetros. Cada uno de estos modelos se evalúa sobre el conjunto de validación, y el que da los mejores resultados (de acuerdo al Evaluator) es seleccionado. 

El modelo final se vuelve a entrenar con todos los datos disponibles.

In [52]:
# usamos el pipeline probado con el algoritmo de clasificación Naive Bayes
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
nb = NaiveBayes(smoothing=1, modelType="multinomial")

pipeline = Pipeline(stages=[countVec, idf, nb])

#
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]) \
    .addGrid(countVec.minTF, [1.0, 3.0, 5.0])\
    .build()
    
#
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)  

In [53]:
# ejecutamos cross-validation
model_eng_cv = crossval.fit(train_english)

In [54]:
model_spa_cv = crossval.fit(train_spanish)

In [55]:
# mostrar los mejores parámetros
mejores_parametros(model_eng_cv, model_spa_cv)

Vectorizer parameters datos inglés:
   CountVectorizer_b616a494e8fb__binary False
   CountVectorizer_b616a494e8fb__maxDF 9.223372036854776e+18
   CountVectorizer_b616a494e8fb__minDF 1.0
   CountVectorizer_b616a494e8fb__minTF 1.0
   CountVectorizer_b616a494e8fb__outputCol countfeatures
   CountVectorizer_b616a494e8fb__vocabSize 262144
   CountVectorizer_b616a494e8fb__inputCol tokens_clean


Vectorizer parameters datos español:
   CountVectorizer_b616a494e8fb__binary False
   CountVectorizer_b616a494e8fb__maxDF 9.223372036854776e+18
   CountVectorizer_b616a494e8fb__minDF 1.0
   CountVectorizer_b616a494e8fb__minTF 1.0
   CountVectorizer_b616a494e8fb__outputCol countfeatures
   CountVectorizer_b616a494e8fb__vocabSize 262144
   CountVectorizer_b616a494e8fb__inputCol tokens_clean


Model parameters datos inglés:
   NaiveBayes_a63bf89f899a__featuresCol features
   NaiveBayes_a63bf89f899a__labelCol label
   NaiveBayes_a63bf89f899a__modelType multinomial
   NaiveBayes_a63bf89f899a__predictionCo

In [56]:
# usamos los modelos para predecir usando los datos de test
pred_eng_cv = model_eng_cv.transform(test_english)
pred_spa_cv = model_spa_cv.transform(test_spanish)

In [57]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_cv, pred_spa_cv)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.734045


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.762048




In [64]:
# ahora probamos el uso de CrossValidator con el algoritmo de LogisticRegression
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
lr = LogisticRegression(maxIter=10, regParam=0.001, elasticNetParam=0)

pipeline_cv_lr = Pipeline(stages=[countVec, idf, lr])

paramGrid_cv_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01, 0.005]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.2])\
    .addGrid(lr.maxIter, [5, 20])\
    .addGrid(countVec.minTF, [1.0, 3.0, 5.0])\
    .build()

crossval_cv_lr = CrossValidator(estimator=pipeline_cv_lr,
                          estimatorParamMaps=paramGrid_cv_lr,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)  

In [65]:
model_eng_cv_lr = crossval_cv_lr.fit(train_english)

In [66]:
model_spa_cv_lr = crossval_cv_lr.fit(train_spanish)

In [67]:
# mostrar los mejores parámetros
mejores_parametros(model_eng_cv_lr, model_spa_cv_lr)

Vectorizer parameters datos inglés:
   CountVectorizer_770593e2c78e__binary False
   CountVectorizer_770593e2c78e__maxDF 9.223372036854776e+18
   CountVectorizer_770593e2c78e__minDF 1.0
   CountVectorizer_770593e2c78e__minTF 1.0
   CountVectorizer_770593e2c78e__outputCol countfeatures
   CountVectorizer_770593e2c78e__vocabSize 262144
   CountVectorizer_770593e2c78e__inputCol tokens_clean


Vectorizer parameters datos español:
   CountVectorizer_770593e2c78e__binary False
   CountVectorizer_770593e2c78e__maxDF 9.223372036854776e+18
   CountVectorizer_770593e2c78e__minDF 1.0
   CountVectorizer_770593e2c78e__minTF 1.0
   CountVectorizer_770593e2c78e__outputCol countfeatures
   CountVectorizer_770593e2c78e__vocabSize 262144
   CountVectorizer_770593e2c78e__inputCol tokens_clean


Model parameters datos inglés:
   LogisticRegression_fcdb86672b4e__aggregationDepth 2
   LogisticRegression_fcdb86672b4e__elasticNetParam 0.1
   LogisticRegression_fcdb86672b4e__family auto
   LogisticRegression_f

In [68]:
# usamos los modelos para predecir usando los datos de test
pred_eng_cv_lr = model_eng_cv_lr.transform(test_english)
pred_spa_cv_lr = model_spa_cv_lr.transform(test_spanish)

In [69]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_cv_lr, pred_spa_cv_lr)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.760526


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.806676




#### Pruebas con los otros datasets cargados desde ficheros csv con datos ya etiquetados.

Vamos a usar los otros datasets cargados desde ficheros csv:
- Con datos en inglés y español con el mismo balance de registros entre las 3 categorías.
- Con datos en inglés y español con el mismo número de registros de solo 2 categorías: negativo y positivo.

In [22]:
# llamamos a la función que engloba el preprocesado de los datos, con la tokenización y la limpieza de tweets
df_tokens_english_clean_neutro, df_tokens_spanish_clean_neutro = preprocesado_dataframe(df_csv_english_neutro,\
                                                                          df_csv_spanish_neutro)

In [23]:
df_tokens_english_clean_neutro = df_tokens_english_clean_neutro.drop("text")
df_tokens_spanish_clean_neutro = df_tokens_spanish_clean_neutro.drop("text")

In [24]:
df_english_clean_neutro = df_tokens_english_clean_neutro.withColumnRenamed("sentiment", "label").cache()
df_spanish_clean_neutro = df_tokens_spanish_clean_neutro.withColumnRenamed("sentiment", "label").cache()

In [25]:
train_english_neutro, test_english_neutro = df_english_clean_neutro.randomSplit([0.9, 0.1], seed=12345)
train_spanish_neutro, test_spanish_neutro = df_spanish_clean_neutro.randomSplit([0.9, 0.1], seed=12345)

In [26]:
train_english_neutro.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|12360|
|    2|12502|
|    0|12552|
+-----+-----+



In [27]:
train_spanish_neutro.select('label').groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 2536|
|    2| 2622|
|    0| 2539|
+-----+-----+



In [85]:
# vamos a lanzar una prueba con crossvalidator y logisticRegression, con los datos que tienen practicamente el mismo
# número de registros de las 3 clases distintas de sentimiento.
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
lr = LogisticRegression(maxIter=10, regParam=0.001, elasticNetParam=0)

pipeline_cv_lr_neutro = Pipeline(stages=[countVec, idf, lr])

paramGrid_cv_lr_neutro = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01, 0.005]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.2])\
    .addGrid(lr.maxIter, [5, 20])\
    .addGrid(countVec.minTF, [1.0, 3.0, 5.0])\
    .build()

crossval_cv_lr_neutro = CrossValidator(estimator=pipeline_cv_lr_neutro,
                          estimatorParamMaps=paramGrid_cv_lr_neutro,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)

In [91]:
model_eng_cv_lr_neutro = crossval_cv_lr_neutro.fit(train_english_neutro)

In [92]:
model_spa_cv_lr_neutro = crossval_cv_lr_neutro.fit(train_spanish_neutro)

In [93]:
# mostrar los mejores parámetros
mejores_parametros(model_eng_cv_lr_neutro, model_spa_cv_lr_neutro)

Vectorizer parameters datos inglés:
   CountVectorizer_9187c9b13a2b__binary False
   CountVectorizer_9187c9b13a2b__maxDF 9.223372036854776e+18
   CountVectorizer_9187c9b13a2b__minDF 1.0
   CountVectorizer_9187c9b13a2b__minTF 1.0
   CountVectorizer_9187c9b13a2b__outputCol countfeatures
   CountVectorizer_9187c9b13a2b__vocabSize 262144
   CountVectorizer_9187c9b13a2b__inputCol tokens_clean


Vectorizer parameters datos español:
   CountVectorizer_9187c9b13a2b__binary False
   CountVectorizer_9187c9b13a2b__maxDF 9.223372036854776e+18
   CountVectorizer_9187c9b13a2b__minDF 1.0
   CountVectorizer_9187c9b13a2b__minTF 1.0
   CountVectorizer_9187c9b13a2b__outputCol countfeatures
   CountVectorizer_9187c9b13a2b__vocabSize 262144
   CountVectorizer_9187c9b13a2b__inputCol tokens_clean


Model parameters datos inglés:
   LogisticRegression_35077ec1faf4__aggregationDepth 2
   LogisticRegression_35077ec1faf4__elasticNetParam 0.2
   LogisticRegression_35077ec1faf4__family auto
   LogisticRegression_3

In [94]:
# usamos los modelos para predecir usando los datos de test
pred_eng_cv_lr_neutro = model_eng_cv_lr_neutro.transform(test_english_neutro)
pred_spa_cv_lr_neutro = model_spa_cv_lr_neutro.transform(test_spanish_neutro)

In [95]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_cv_lr_neutro, pred_spa_cv_lr_neutro)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.590517


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.625248




#### Prueba con TrainValidationSplit en lugar de CrossValidation.

In [28]:
# probamos el modelo con los mejores resultados, y el uso de TrainValidationSplit en lugar de CrossValidation
countVec = CountVectorizer(inputCol="tokens_clean", outputCol="countfeatures")
idf = IDF(inputCol=countVec.getOutputCol(), outputCol="features", minDocFreq=2)
lr = LogisticRegression(maxIter=10, regParam=0.001, elasticNetParam=0)

pipeline_cv_lr = Pipeline(stages=[countVec, idf, lr])

paramGrid_cv_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01, 0.005]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.2])\
    .addGrid(lr.maxIter, [5, 20])\
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(countVec.minTF, [1.0, 3.0, 5.0])\
    .build()

tvs = TrainValidationSplit(estimator=pipeline_cv_lr,
                           estimatorParamMaps=paramGrid_cv_lr,
                           evaluator=MulticlassClassificationEvaluator(),
                           trainRatio=0.9)

In [29]:
model_tvs_eng = tvs.fit(train_english)

In [30]:
model_tvs_spa = tvs.fit(train_spanish)

In [31]:
# usamos los modelos para predecir usando los datos de test
pred_eng_tvs = model_tvs_eng.transform(test_english)
pred_spa_tvs = model_tvs_spa.transform(test_spanish)

In [32]:
# evaluamos el modelo
evaluar_modelo('f1', 'prediction', 'label', pred_eng_tvs, pred_spa_tvs)

-------------------------------------------------------
Medidas de rendimiento datos en inglés
-------------------------------------------------------
F1-score = 0.761035


-------------------------------------------------------
Medidas de rendimiento datos en español
-------------------------------------------------------
F1-score = 0.801400




#### Probar el modelo para predecir los tweets sin etiquetar recogidos de MongoDB.

Usando el modelo anterior, donde se han obtenido más o menos los mejores resultados que con el mejor anterior (sobre el 76% con datos en inglés, y un 80% con datos en español, vamos a predecir el sentimiento de los datos almacenados en MongoDB.

In [33]:
# cogemos un sample del DF con datos en inglés ya que tiene muchos datos y da problemas de cómputo y tiempos
df_mongo_english_sample = df_mongo_english.sample(False, 0.4, 45)

In [34]:
# llamamos a la función que engloba el preprocesado de los datos, con la tokenización y la limpieza de tweets
df_tokens_english_clean_mongo, df_tokens_spanish_clean_mongo = preprocesado_dataframe(df_mongo_english_sample,\
                                                                          df_mongo_spanish)

In [35]:
df_tokens_english_clean_mongo.limit(10).toPandas()

,text,tokens_clean
0,RT @liamyoung: Strange that Tony Blair has sud...,"[strange , tony , blair , suddenly , become , ..."
1,Hard work puts you where good luck can find you.,"[hard , work , puts , good , luck , find ]"
2,RT @xCiphxr: When creative kids try playing co...,"[creative , kids , try , playing , comp ]"
3,RT @bonang_m: I’m working on one as we speak. ...,"[im , working , one , speak , thank , incredib..."
4,"RT @akashbanerjee: After #PulwamaAttack, terro...","[pulwamaattack , terrorists , scored , bigger ..."
5,RT @godisgodisback: Beyoncé: WORLD STOP!\n\nWo...,"[beyonc , world , stop , world ]"
6,No. It just needs money and connections. Your ...,"[needs , money , connections , average , norma..."
7,RT @hopeoverfear01: It’s time to run our own c...,"[time , run , country , retweet , agree ]"
8,RT @gugugaga__: this works. i’m not even gonna...,"[works , im , even , gon na , front , retweete..."
9,RT @pushetblur: Trying to prove a point. Lets ...,"[trying , prove , point , lets , settle , retw..."


In [37]:
df_tokens_spanish_clean_mongo.limit(10).toPandas()

,text,tokens_clean
0,@EJFC26 Y Messi,[messi ]
1,RT @MBelenAlegre: Hermoso y sensual viernes ❣️,"[hermoso , sensual , viernes ]"
2,RT @Foro_TV: Suman 47 muertos y 640 heridos po...,"[suman , muertos , heridos , explosion , china ]"
3,RT @revistaetcetera: .@lopezobrador_ dice que ...,"[dice , tan , grande , gloria , benito , juare..."
4,"Me retracto de mi respuesta , él gobierno se s...","[retracto , respuesta , gobierno , acorralado ]"
5,Anda a saber si te esta eligiendo o sos lo úni...,"[anda , saber , si , eligiendo , sos , unico ,..."
6,RT @JCTrujilloCPCCS: Hay que propiciar una Con...,"[propiciar , consulta , popular , pueblo , ecu..."
7,@RicciuP Terraplanistas,[terraplanistas ]
8,RT @oriolguellipuig: Jeje jeje https://t.co/NU...,"[jeje , jeje ]"
9,quienes son las veganarcas? no quiero ser part...,"[veganarcas , quiero , ser , parte , team , ex..."


In [38]:
# usamos los modelos para predecir usando los datos de test
pred_eng_clean = model_tvs_eng.transform(df_tokens_english_clean_mongo)
pred_spa_clean = model_tvs_spa.transform(df_tokens_spanish_clean_mongo)

In [41]:
# nos quedamos con las columnas del texto, tokens y la predicción calculada
pred_eng_clean = pred_eng_clean.drop("countfeatures","features","rawPrediction","probability")
pred_spa_clean = pred_spa_clean.drop("countfeatures","features","rawPrediction","probability")

In [42]:
print("Predictions:")
pred_eng_clean.limit(20).toPandas()

Predictions:


,text,tokens_clean,prediction
0,RT @liamyoung: Strange that Tony Blair has sud...,"[strange , tony , blair , suddenly , become , ...",0.0
1,Hard work puts you where good luck can find you.,"[hard , work , puts , good , luck , find ]",0.0
2,RT @xCiphxr: When creative kids try playing co...,"[creative , kids , try , playing , comp ]",2.0
3,RT @bonang_m: I’m working on one as we speak. ...,"[im , working , one , speak , thank , incredib...",2.0
4,"RT @akashbanerjee: After #PulwamaAttack, terro...","[pulwamaattack , terrorists , scored , bigger ...",2.0
5,RT @godisgodisback: Beyoncé: WORLD STOP!\n\nWo...,"[beyonc , world , stop , world ]",2.0
6,No. It just needs money and connections. Your ...,"[needs , money , connections , average , norma...",0.0
7,RT @hopeoverfear01: It’s time to run our own c...,"[time , run , country , retweet , agree ]",2.0
8,Do me a favor right? Follow this absolute top ...,"[favor , right , follow , absolute , top , fuc...",2.0
9,RT @pushetblur: Trying to prove a point. Lets ...,"[trying , prove , point , lets , settle , retw...",2.0


In [44]:
print("Predictions:")
pred_spa_clean.limit(20).toPandas()

Predictions:


,text,tokens_clean,prediction
0,@EJFC26 Y Messi,[messi ],2.0
1,RT @MBelenAlegre: Hermoso y sensual viernes ❣️,"[hermoso , sensual , viernes ]",2.0
2,RT @Foro_TV: Suman 47 muertos y 640 heridos po...,"[suman , muertos , heridos , explosion , china ]",0.0
3,RT @revistaetcetera: .@lopezobrador_ dice que ...,"[dice , tan , grande , gloria , benito , juare...",0.0
4,"Me retracto de mi respuesta , él gobierno se s...","[retracto , respuesta , gobierno , acorralado ]",0.0
5,Anda a saber si te esta eligiendo o sos lo úni...,"[anda , saber , si , eligiendo , sos , unico ,...",0.0
6,RT @JCTrujilloCPCCS: Hay que propiciar una Con...,"[propiciar , consulta , popular , pueblo , ecu...",2.0
7,@RicciuP Terraplanistas,[terraplanistas ],0.0
8,RT @oriolguellipuig: Jeje jeje https://t.co/NU...,"[jeje , jeje ]",2.0
9,quienes son las veganarcas? no quiero ser part...,"[veganarcas , quiero , ser , parte , team , ex...",2.0
